# Show how extinction curves work with sliders
<em> Just a hack! Don't trust this for your dust!

In [1]:
import numpy as np
import pandas as pd
from bokeh.io import output_file, show, push_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, Select
from bokeh.plotting import Figure, output_file, show, output_notebook
from bokeh.models import Arrow, VeeHead

output_file("extinction_models.html")
output_notebook()


Loading BokehJS ...

In [2]:
# I ran   conda install -c bokeh flexx    first


In [3]:
run getFMext

In [4]:
run getMags_ext

In [5]:
x=np.arange(0.45,5.,.01)
invx=1./x


In [6]:
# make a multi-layer pandas frame... fun, but not useful here

# cols = [np.array(['', '', '', 'R2', 'R2', 'R2', 'R3', 'R3', 'R3', 'R4', 'R4', 'R4', 'R5', 'R5', 'R5']),
#         np.array(['A', 'EHK', 'EJH', 'A', 'EHK', 'EJH', 'A', 'EHK', 'EJH', 'A', 'EHK', 'EJH', 'A', 'EHK', 'EJH'])]

# df = pd.DataFrame(index=x, columns=cols)

# for model in ['f99 tables']:
#     for Rlabel,Rv in [('R2',2.1),('R3',3.1),('R4',4.1)]:
#         df[Rlabel, 'A']=getFMext(x,Rv,model)/Rv
#         m=getMags_ext(x,getFMext(x,Rv,model)/Rv)
#         df[Rlabel, 'EHK']=m['H']-m['Ks']
#         df[Rlabel, 'EJH']=m['J']-m['H']
        

In [7]:
cols=['x','invx']


for Rlabel in ['','R2','R3','R4','R5']:
    for y in ['A','EHK','EJH', 'EVR', 'AV']:
        cols.append(Rlabel+y)    

df = pd.DataFrame(index=x, columns=cols)
df['x']=x
df['invx']=1./x


for model in [('fmunred')]:
    for Rlabel,Rv in [('R2',2.1),('R3',3.1),('R4',4.1),('R5',5.1)]:
        df[Rlabel+'A']=getFMext(x,Rv,model)/Rv
        m=getMags_ext(x,getFMext(x,Rv,model)/Rv)
        df[Rlabel+'EHK']=m['H']-m['Ks']
        df[Rlabel+'EJH']=m['J']-m['H']
        df[Rlabel+'EVR']=m['V']-m['R']
        df[Rlabel+'AV']=m['V']

        
# initialize
df['A']=df['R3A']
df['EJH']=df['R3EJH']
df['EHK']=df['R3EHK']
df['EVR']=df['R3EVR']
df['AV']=df['R3AV']



In [8]:
# Not using the column data source...

# plot1 = Figure(width=300, height=300, x_range=[0,5], y_range=[0,2],
#                x_axis_label='lambda (microns)', y_axis_label='A(lambda)', toolbar_location=None)
# plot1.line(df.index.values, df['R3A'].values, line_width=3, line_color='red')
# 
# show(column(plot1))


In [9]:
# Using the column data source

source=ColumnDataSource(df)



In [10]:
plot1 = Figure(width=300, height=300, x_range=[0.4,5], y_range=[0,2],
               x_axis_label='lambda (microns)', y_axis_label='A(lambda)')
r1 = plot1.line('x', 'A', source=source, line_width=3, line_color='black')

plot2 = Figure(width=300, height=300, x_range=[0,2.5], y_range=[0,2], 
               x_axis_label='1/lambda (1/microns)', y_axis_label='A(lambda)')
r2 = plot2.line('invx', 'A', source=source, line_width=3, line_color='black')

plot3 = Figure(width=300, height=300, x_range=[0,.5], y_range=[0,.5],
               x_axis_label='E(H-Ks)', y_axis_label='E(J-H)')
plot3.segment(0, 0, 'EHK', 'EJH', source=source, line_width=3, line_color='red')
plot3.add_layout(Arrow(end=VeeHead(fill_color='red',line_color='red'), line_color="red",
                       x_start=0, y_start=0, x_end='EHK', y_end='EJH', source=source))

plot4 = Figure(width=300, height=300, x_range=[0,1.], y_range=[0,5.],
               x_axis_label='E(V-R)', y_axis_label='A(V)')
plot4.segment(0, 0, 'EVR', 'AV', source=source, line_width=3, line_color='blue')
plot4.add_layout(Arrow(end=VeeHead(fill_color='blue',line_color='blue'), line_color="blue",
                       x_start=0, y_start=0, x_end='EVR', y_end='AV', source=source))


# show(row(plot1, plot2, plot3))

In [11]:
slider1 = Slider(start=1., end=5., value=1., step=.1, title="A(V)")
slider2 = Slider(start=2.1, end=5.1, value=3.1, step=1., title="R(V)")

def update(source=source, slider1=slider1, slider2=slider2, window=None):
    data = source.data
    y = data['A']
    y1 = data['R3A']
    ejh = data['EJH']
    ehk = data['EHK']
    ejh1 = data['R3EJH']
    ehk1 = data['R3EHK']
    evr = data['EVR']
    evr1 = data['R3EVR']
    evr = data['EVR']
    evr1 = data['R3EVR']
    av = data['AV']
    av1 = data['R3AV']
    
    if slider2.value==2.1:
        y1 = data['R2A']
        ejh1 = data['R2EJH']
        ehk1 = data['R2EHK']
        evr1 = data['R2EVR']
        av1 = data['R2AV']
    if slider2.value==4.1:
        y1 = data['R4A']
        ejh1 = data['R4EJH']
        ehk1 = data['R4EHK']
        evr1 = data['R4EVR']
        av1 = data['R4AV']
    if slider2.value>4.1:
        y1 = data['R5A']
        ejh1 = data['R5EJH']
        ehk1 = data['R5EHK']
        evr1 = data['R5EVR']
        av1 = data['R5AV']
    for i in range(len(y)):
        y[i] = slider1.value*y1[i]
        ehk[i] = slider1.value*ehk1[i]
        ejh[i] = slider1.value*ejh1[i]
        evr[i] = slider1.value*evr1[i]
        av[i] = slider1.value*av1[i]
    source.trigger('change')

    
slider1.js_on_change('value', CustomJS.from_py_func(update))
slider2.js_on_change('value', CustomJS.from_py_func(update))


model = Select(title='Model', value='fmunred', options=['fmunred'] + ['coming soon!'])


t=show(column(row(column(slider1, slider2),model), row(plot1, plot2), row(plot3, plot4)),notebook_handle=True)


In [12]:
# Would be useful to have an output of vector length and slope (R)